ジュピターで画像だけを先にロードして学習だけ書き換えることができるかのテスト

In [1]:
import os
from os.path import join
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import matplotlib.pyplot as plt

from PIL import Image

In [2]:
## 変数群　この辺変えて
size_batch = 1

img_w = 16
img_h = 16

# 学習データの学習回数
n_epoch = 20000

In [3]:
#学習用の画像のロード

#画像を取得 return 画像のテンソル, 画像のラベル番号
def get_img(path):
    img = Image.open(path)
    img = np.array(img, dtype=np.float32)
    return img

#バッチ作成
def make_batch(list_path_img):
    x_batch = []
    t_batch = []
    for path_img in list_path_img:
        img = get_img(path_img)
        img = np.array(img, dtype=np.float32)
        img = img.transpose(2, 0, 1)
        x_batch.append(img)
        a = str(path_img).split('/')[-2]
        a = int(a.split("_")[-1])
        t_batch.append(a)
        #print(a)
    return torch.tensor(x_batch), torch.tensor(t_batch)


train_list = []
x_batch = [[] for i in range(n_epoch)]
t_batch = [[] for i in range(n_epoch)]

for batch_number in range(n_epoch):
    for k in range(img_h):
        for m in range(img_w):
            train_list.append(f"C:/procon_data/data/16x16_v1/({k}, {m})_{k*img_w + m}/{batch_number}.jpg")
        x_batch[batch_number], t_batch[batch_number] = make_batch(train_list)
        train_list.clear()

NameError: name 'get_img' is not defined

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

#NNモデル
class MyNet2(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(16*16*3,768)# Input Layer to Intermediate modules
        self.fc2 = nn.Linear(768,768) #Intermediate modules to Output Layer
        self.fc3 = nn.Linear(768,1024) #Intermediate modules to Output Layer
        self.fc4 = nn.Linear(1024,1024) #Intermediate modules to Output Layer
        self.fc5 = nn.Linear(1024,1024) #Intermediate modules to Output Layer
        self.fc6 = nn.Linear(1024,512)
        self.fc7 = nn.Linear(512,512)
        self.fc8 = nn.Linear(512,512)
        self.fc9 = nn.Linear(512,256)
        self.fca = nn.Linear(256,256)
        self.fcb = nn.Linear(256,256)
        self.fcc = nn.Linear(256,256)
        self.bn0 = nn.BatchNorm2d(num_features=16) 

    def forward(self, x):#順伝播 Forward propagation
        x = x.view(-1,16*16*3)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        x = self.fc4(x)
        x = self.fc5(x)
        x = self.fc6(x)
        x = self.fc7(x)
        x = self.fc8(x)
        x = self.fc9(x)
        x = self.fca(x)
        x = self.fcb(x)
        y = self.fcc(x)
        #y = nn.Softmax(self.fcp(x))
        return y


#モデルに画像を入力し学習させる return なし

mymodel = MyNet2().to(device)
opt = optim.Adam(mymodel.parameters(), lr=0.001)
print(torch.cuda.get_device_name())

plot_x = []
# ロスと精度を保存するリスト（訓練用・テスト用）
list_loss_train = []
list_loss_test = []
list_acc_train = []
list_acc_test = []

def reset():
    new_model = MyNet2()
    return new_model


a
GeForce RTX 2060


In [ ]:
i = 0
temp_model = mymodel
while i < n_epoch-1:#最後のやつは評価用
    temp_max = 0
    if i%1000 == 0 and i != 0:
        #1000回やって正答率5%以下ならやり直し
        print("1000time")
        if max(list_acc_train[i-1000:]) < 0.03 or max(list_acc_train[i-1000:]) < temp_max:
            if i == 1000:
                mymodel = reset()
                mymodel.to(device)
                i = 0
            else:
                mymodel = temp_model
            print("reset")
            print(temp_model == mymodel)
        else:
            temp_max = max(list_acc_train[i-1000:])
            temp_mpdel  = mymodel
            print(f"nice {max(list_acc_train[i-1000:])}")
    
    sum_loss = 0.
    sum_acc = 0.
    opt.zero_grad()
    x = x_batch[i].to(device)
    t = t_batch[i].to(device)
    y = mymodel(x)
    loss = F.cross_entropy(y, t)

    # 逆伝播
    
    loss.backward()

    

    # ロスと精度を蓄積
    sum_loss += loss.item()
    sum_acc += (y.max(1)[1] == t).sum().item()

    # パラメータ更新
    opt.step()
    #scheduler.step()

    mean_loss = sum_loss / len(x)
    mean_acc = sum_acc / len(x)
    list_loss_train.append(mean_loss)
    list_acc_train.append(mean_acc)
    plot_x.append(i)
    print(f"- now {i}")
    print("- mean loss:", mean_loss)
    print("- mean accuracy:", mean_acc) 
    i += 1
      

print("fin v5 both steped")
print("haha")
print(list_acc_train)
print(f"max acc {max(list_acc_train)}")
print(f"-mean acc {sum(list_acc_train)/len(list_acc_train)}")
plt.plot(plot_x,list_acc_train)

- now 0
- mean loss: 0.02172500640153885
- mean accuracy: 0.00390625
- now 1
- mean loss: 0.02313360571861267
- mean accuracy: 0.0
- now 2
- mean loss: 0.05432939901947975
- mean accuracy: 0.00390625
- now 3
- mean loss: 0.03593531250953674
- mean accuracy: 0.00390625
- now 4
- mean loss: 0.038800790905952454
- mean accuracy: 0.00390625
- now 5
- mean loss: 0.04579285904765129
- mean accuracy: 0.00390625
- now 6
- mean loss: 0.08561109006404877
- mean accuracy: 0.00390625
- now 7
- mean loss: 0.10734203457832336
- mean accuracy: 0.00390625
- now 8
- mean loss: 0.05738551542162895
- mean accuracy: 0.00390625
- now 9
- mean loss: 0.2653394341468811
- mean accuracy: 0.00390625
- now 10
- mean loss: 0.14502637088298798
- mean accuracy: 0.00390625
- now 11
- mean loss: 0.07477810233831406
- mean accuracy: 0.00390625
- now 12
- mean loss: 0.28518521785736084
- mean accuracy: 0.00390625
- now 13
- mean loss: 0.09897390007972717
- mean accuracy: 0.00390625
- now 14
- mean loss: 0.0287405103445

In [ ]:
x_e = x_batch[-1].to(device)
t_e = t_batch[-1].to(device)
y = mymodel(x)
loss = F.cross_entropy(y_e, t_e)

# 逆伝播

loss.backward()



# ロスと精度を蓄積
sum_loss += loss.item()
sum_acc += (y.max(1)[1] == t).sum().item()

#scheduler.step()

mean_loss = sum_loss / len(x)
mean_acc = sum_acc / len(x)
list_loss_train.append(mean_loss)
list_acc_train.append(mean_acc)
plot_x.append(i)
print(f"- now {i}")
print("- mean loss:", mean_loss)
print("- mean accuracy:", mean_acc) 